In [1]:
import evebox as box
import pandas as pd
import tensorflow as tf
import cProfile
from tqdm.notebook import tqdm, trange

In [2]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.5).with_market_types()

In [3]:
#orders = box.load_orders(uni, tqdm = tqdm)
orders = pd.read_csv('market/current.csv')

In [15]:
#orders.to_csv('market/current.csv')

In [4]:
orders

,order_id,duration,is_buy_order,issued,location_id,min_volume,price,range,system_id,type_id,volume_remain,volume_total
0,911190994,365,False,2020-03-09T13:11:57+00:00,60000004,1,15.05,region,30002780,41,23572,23572
1,911190995,365,False,2020-03-09T13:11:57+00:00,60000007,1,14.03,region,30002779,41,23572,23572
2,911190996,365,False,2020-03-09T13:11:57+00:00,60000010,1,15.05,region,30002776,41,23572,23572
3,911190997,365,False,2020-03-09T13:11:57+00:00,60000013,1,14.03,region,30002779,41,23572,23572
4,911190998,365,False,2020-03-09T13:11:57+00:00,60000016,1,14.95,region,30002778,41,23572,23572
...,...,...,...,...,...,...,...,...,...,...,...,...
892078,5629348134,1,True,2020-03-09T13:26:22+00:00,60012103,1,11360.00,5,30005316,35660,1,1
892079,5629348393,1,False,2020-03-09T13:26:42+00:00,60015020,1,1227777.81,region,30045042,482,1,1
892080,5629348404,1,True,2020-03-09T13:26:44+00:00,60012103,1,11360.00,5,30005316,35660,1,1
892081,5629348535,1,False,2020-03-09T13:26:56+00:00,60015020,1,1227777.81,region,30045042,482,1,1


In [4]:
gym = box.tf.TradingGym(uni, orders)

In [5]:
jita = [s["system_id"] for s in uni.systems.values() if s["name"] == "Jita"][0]

state = box.MutableState()
state.time_left = 20.0
state.system    = jita

state = box.State(state)

In [6]:
class Model(tf.keras.Model):
    def __init__(self, universe):
        super().__init__()
        
        self.bandwidth = 128
        self.d_notes  = 4
        
        self.embedding = box.tf.Embedding(universe, d_notes = self.d_notes, tqdm = tqdm)
        
        self.input_transforms = {
            k : tf.keras.layers.Dense(self.bandwidth)
            for k in ['state', 'orders', 'cargo', 'systems']
        }
        
        def otf(n):
            return tf.keras.layers.Dense(n)
        
        self.output_transforms = {
            'actions' : otf(3),
            'move_targets' : otf(1),
            'buy_params' : otf(3),
            'sell_params' : otf(3),
            'value' : otf(1)
        }
        
        self.stack_1 = box.tf.StackedTogether(
            tf.keras.Sequential([
                box.tf.MultiHeadedAttention(self.bandwidth, 16, 8) for i in range(2)
            ])
        )
        
        self.stack_2 = box.tf.StackedTogether(
            tf.keras.Sequential([])
        )
    
    def get_initial_state(self, input):
        batch_shape = tf.shape(input['state'])[:-1]
        state_shape = tf.concat([batch_shape, [self.bandwidth]], axis = 0)
        
        return tf.zeros(state_shape, dtype = tf.float32)
    
    def call(self, input):
        input, rnn_state = input
        
        # Pre-process input (adds "note" variables to types & systems, joins orders with their types & systems)
        input = self.embedding(input)
        
        # Expand state to have the same shape as the other stuff
        input['state'] = tf.expand_dims(input['state'], axis = -2)
        
        # Expand all items into a [...,128] shape
        input = {
            k : self.input_transforms[k](v)
            for k, v in input.items()
        }
        
        # Add RNN state to the mix
        input['rnn_state'] = tf.expand_dims(rnn_state, axis = -2)
        
        # Run 2 model stacks, one with and one without orders
        input = self.stack_1(input)
        del input['orders']
        input = self.stack_2(input)
        
        rnn_state = tf.squeeze(input['rnn_state'], axis = -2)
        del input['rnn_state']
        
        output = {
            'actions' : tf.squeeze(
                self.output_transforms['actions'](input['state']),
                axis = -2
            ),
            
            'move_targets' : tf.squeeze(
                self.output_transforms['move_targets'](input['systems']),
                axis = -1
            ),
            
            'buy_params'  : self.output_transforms['buy_params'] (input['cargo']),
            'sell_params' : self.output_transforms['sell_params'](input['cargo']),
            
            'value' : tf.squeeze(
                self.output_transforms['value'](input['state']),
                axis = [-2, -1]
            )
        }
        
        return output, rnn_state
        
        
        

model = Model(uni)

In [7]:
unroller = gym.unroll_model(model, tqdm);

opt = tf.keras.optimizers.SGD(1e-4)

unroller(state, 1)

def get_loss():
    result = unroller(state, 20)
    
    policy_loss, value_loss, entropy_loss = gym.losses(result)
    tf.print(policy_loss)
    tf.print(value_loss)
    tf.print(entropy_loss)
    return entropy_loss

for i in range(5):
    opt.minimize(get_loss, model.trainable_variables)

W0309 14:47:21.776231  6368 backprop.py:866] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0309 14:47:21.776231  6368 backprop.py:866] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0309 14:47:22.769773  6368 backprop.py:1003] The dtype of the target tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int32
W0309 14:47:22.769773  6368 backprop.py:1003] The dtype of the target tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int32
W0309 14:47:22.769773  6368 backprop.py:1003] The dtype of the target tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int32
W0309 14:47:22.769773  6368 backprop.py:1017] The dtype of the source tensor must be floating (e.g. tf.float32) when calling GradientTape.gradient, got tf.int32
W0309 14:47:22.769773  6368 backprop.py:1017] The dtype of the source tensor must be floating (e.g. tf.f


-17.5010872
0.624991655
-19.9233894


W0309 14:48:57.950043  6368 optimizer_v2.py:1043] Gradients do not exist for variables ['model/stacked_together/sequential/multi_headed_attention/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/bias:0'] when minimizing the loss.



-27.5150986
0.89475441
-23.217474


W0309 14:50:32.965917  6368 optimizer_v2.py:1043] Gradients do not exist for variables ['model/stacked_together/sequential/multi_headed_attention/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/bias:0'] when minimizing the loss.


...

W0309 14:52:08.697642  6368 optimizer_v2.py:1043] Gradients do not exist for variables ['model/stacked_together/sequential/multi_headed_attention/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/bias:0'] when minimizing the loss.



-34.3979
1.26555979
-22.2342377


W0309 14:53:51.132186  6368 optimizer_v2.py:1043] Gradients do not exist for variables ['model/stacked_together/sequential/multi_headed_attention/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/bias:0'] when minimizing the loss.



-27.0939255
0.912451088
-22.2103138


W0309 14:55:36.106258  6368 optimizer_v2.py:1043] Gradients do not exist for variables ['model/stacked_together/sequential/multi_headed_attention/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention/Wv/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wk/bias:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/kernel:0', 'model/stacked_together/sequential/multi_headed_attention_1/Wv/bias:0'] when minimizing the loss.


In [13]:
states, actions, logps, values = zip(*result)

actions

(Buy 61411932.0 of 42241,
 Buy 10420804.0 of 11567,
 Buy 41306360.0 of 11567,
 Buy 56765748.0 of 11567,
 Buy 25071562.0 of 42241,
 Buy 71965600.0 of 42241,
 Buy 69669368.0 of 42241,
 Buy 70687472.0 of 11567,
 Buy 44765412.0 of 42241,
 Buy 31007356.0 of 42241)

In [16]:
uni.types[11567]

{'capacity': 11250.0,
 'description': 'Casting his sight on his realm, the Lord witnessed\r\nThe cascade of evil, the torrents of war.\r\nBurning with wrath, He stepped \r\ndown from the Heavens\r\nTo judge the unworthy,\r\nTo redeem the pure.\r\n\r\n<i>-The Scriptures, Revelation Verses 2:12</i>\r\n\r\n',
 'dogma_attributes': [{'attribute_id': 3, 'value': 0.0},
  {'attribute_id': 4, 'value': 2400000000.0},
  {'attribute_id': 9, 'value': 390000.0},
  {'attribute_id': 11, 'value': 1250000.0},
  {'attribute_id': 12, 'value': 8.0},
  {'attribute_id': 13, 'value': 5.0},
  {'attribute_id': 14, 'value': 8.0},
  {'attribute_id': 15, 'value': 0.0},
  {'attribute_id': 18, 'value': 0.0},
  {'attribute_id': 19, 'value': 1.0},
  {'attribute_id': 21, 'value': 0.0},
  {'attribute_id': 37, 'value': 60.0},
  {'attribute_id': 38, 'value': 11250.0},
  {'attribute_id': 552, 'value': 22760.0},
  {'attribute_id': 48, 'value': 815.0},
  {'attribute_id': 49, 'value': 0.0},
  {'attribute_id': 564, 'value': 70

In [8]:
losses = gym.losses(result)

In [9]:
losses

(<tf.Tensor: shape=(), dtype=float32, numpy=2.7522936>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0017755391>,
 <tf.Tensor: shape=(), dtype=float32, numpy=217.35147>)

In [ ]:
with tf.GradientTape() as tape:
    result = unroller(
        state, 150
    )
    #cProfile.run('unroller(state, 100)', 'profile_stats')

In [ ]:
import pstats
p = pstats.Stats('profile_stats')
p.sort_stats('cumtime')
p.print_stats()

In [ ]:
result[0:1]

In [ ]:
tape.gradient(result[0][2], sources=model.trainable_variables)

In [ ]:
model.trainable_variables